# LLM Strict JSON Tabular Data Interpolator
- Created by John Tan Chong Min
- https://github.com/tanchongmin/strictjson
- 21 Aug 2023
- Trying to see how to adapt Strict JSON Framework for Tabular Data

In [146]:
import os
import openai
import json
import re

#API Keys
os.environ['OPENAI_API_TOKEN'] = 'YOUR_API_KEY_HERE'
openai.api_key = os.environ['OPENAI_API_TOKEN']

# Strict Output Formatting
- Use when you want to force the function output to be a json format
- Helps a lot with minimizing unnecessary explanations of ChatGPT, and ensuring all output fields are there

In [148]:
def strict_output(system_prompt, user_prompt, output_format, default_category = "", output_value_only = False,
                  model = 'gpt-3.5-turbo', temperature = 0, num_tries = 2, verbose = False):
    ''' Ensures that OpenAI will always adhere to the desired output json format. 
    Uses rule-based iterative feedback to ask GPT to self-correct.
    Keeps trying up to num_tries it it does not. Returns empty json if unable to after num_tries iterations.
    If output field is a list, will treat as a classification problem and output best classification category.
    Text enclosed within < > will generated by GPT accordingly'''

    # if the user input is in a list, we also process the output as a list of json
    list_input = isinstance(user_prompt, list)
    # if the output format contains dynamic elements of < or >, then add to the prompt to handle dynamic elements
    dynamic_elements = '<' in str(output_format)
    # if the output format contains list elements of [ or ], then we add to the prompt to handle lists
    list_output = '[' in str(output_format)
    
    # start off with no error message
    error_msg = ''
    
    for i in range(num_tries):
        
        output_format_prompt = f'''\nYou are to output the following in json format: {output_format}. 
Do not put quotation marks or escape character \ in the output fields.'''
        
        if list_output:
            output_format_prompt += f'''\nIf output field is a list, classify output into the best element of the list.'''
        
        # if output_format contains dynamic elements, process it accordingly
        if dynamic_elements: 
            output_format_prompt += f'''
Any text enclosed by < and > indicates you must generate content to replace it. Example input: Go to <location>, Example output: Go to the garden
Any output key containing < and > indicates you must generate the key name to replace it. Example input: {{'<location>': 'description of location'}}, Example output: {{'school': 'a place for education'}}'''

        # if input is in a list format, ask it to generate json in a list
        if list_input:
            output_format_prompt += '''\nGenerate a list of json, one json for each input element.'''
            
        # Use OpenAI to get a response
        response = openai.ChatCompletion.create(
          temperature = temperature,
          model=model,
          messages=[
            {"role": "system", "content": system_prompt + output_format_prompt + error_msg},
            {"role": "user", "content": str(user_prompt)}
          ]
        )

        res = response['choices'][0]['message']['content'].replace('\'', '"')
        
        # ensure that we don't replace away aprostophes in text 
        res = re.sub(r"(\w)\"(\w)", r"\1'\2", res)

        if verbose:
            print('System prompt:', system_prompt + output_format_prompt + error_msg)
            print('\nUser prompt:', str(user_prompt))
            print('\nGPT response:', res)
        
        # try-catch block to ensure output format is adhered to
        try:
            output = json.loads(res)
            if isinstance(user_prompt, list):
                if not isinstance(output, list): raise Exception("Output format not in a list of json")
            else:
                output = [output]
                
            # check for each element in the output_list, the format is correctly adhered to
            for index in range(len(output)):
                for key in output_format.keys():
                    # unable to ensure accuracy of dynamic output header, so skip it
                    if '<' in key or '>' in key: continue
                    # if output field missing, raise an error
                    if key not in output[index]: raise Exception(f"{key} not in json output")
                    # check that one of the choices given for the list of words is an unknown
                    if isinstance(output_format[key], list):
                        choices = output_format[key]
                        # ensure output is not a list
                        if isinstance(output[index][key], list):
                            output[index][key] = output[index][key][0]
                        # output the default category (if any) if GPT is unable to identify the category
                        if output[index][key] not in choices and default_category:
                            output[index][key] = default_category
                        # if the output is a description format, get only the label
                        if ':' in output[index][key]:
                            output[index][key] = output[index][key].split(':')[0]
                            
                # if we just want the values for the outputs
                if output_value_only:
                    output[index] = [value for value in output[index].values()]
                    # just output without the list if there is only one element
                    if len(output[index]) == 1:
                        output[index] = output[index][0]
                    
            return output if list_input else output[0]

        except Exception as e:
            error_msg = f"\n\nResult: {res}\n\nError message: {str(e)}"
            print("An exception occurred:", str(e))
            print("Current invalid json format:", res)
         
    return {}

## Overall Open-ended generation
- **system_prompt**: Write in whatever you want GPT to become. "You are a \<purpose in life\>"
- **user_prompt**: The user input. Later, when we use it as a function, this is the function input
- **output_format**: JSON format with the key as the output key, and the value as the output description
    - The output keys will be preserved exactly, while GPT will generate content to match the description of the value as best as possible

#### Example Usage
```python
res = strict_output(system_prompt = 'You are a classifier',
                    user_prompt = 'It is a beautiful day',
                    output_format = {"Sentiment": "Type of Sentiment",
                                    "Tense": "Type of Tense"})
                                    
print(res)
```

#### Example output
```{'Sentiment': 'Positive', 'Tense': 'Present'}```


## Generating more tabular data

In [308]:
import pandas as pd

# Read the CSV file
# df = pd.read_csv('jobs_data.csv')
start_info = [{'Name': 'John', 'Gender': 'Male', 'Schwag': 'Yes', 'Occupation': 'NaN', 'Occupation Description': 'Works to develop Large Language Models', 'Salary': 'NaN'}, 
{'Name': 'NaN', 'Gender': 'Female', 'Schwag': 'No', 'Occupation': 'Nurse', 'Occupation Description': 'NaN', 'Salary': '5000'}, 
{'Name': 'Kevin', 'Gender': 'NaN', 'Schwag': 'Yes', 'Occupation': 'Founder', 'Occupation Description': 'NaN', 'Salary': 'NaN'}]
df = pd.DataFrame(start_info)

# Display the DataFrame
df

,Name,Gender,Schwag,Occupation,Occupation Description,Salary
0,John,Male,Yes,NaN,Works to develop Large Language Models,NaN
1,NaN,Female,No,Nurse,NaN,5000
2,Kevin,NaN,Yes,Founder,NaN,NaN


# Data Imputation
- impute data
- generate a few more rows

In [287]:
res = strict_output(system_prompt = f'''
You are given rows of Tabular Data with missing values denoted as NaN. 
You must not truncate the data.
''', 
        user_prompt = df,
        output_format = 
{"Filled Data": "<generate missing values for the csv data in the same format as given>",
"Generated Data": "<generate another 3 more rows of data based on the original data>"})

print(res)
my_df = pd.DataFrame(res['Filled Data']+res['Generated Data'])

{'Filled Data': [{'Name': 'John', 'Gender': 'Male', 'Schwag': 'Yes', 'Occupation': 'Developer', 'Occupation Description': 'Works to develop Large Language Models', 'Salary': '10000'}, {'Name': 'Jane', 'Gender': 'Female', 'Schwag': 'No', 'Occupation': 'Nurse', 'Occupation Description': 'Takes care of patients', 'Salary': '5000'}, {'Name': 'Kevin', 'Gender': 'Male', 'Schwag': 'Yes', 'Occupation': 'Founder', 'Occupation Description': 'Starts and runs a business', 'Salary': '20000'}], 'Generated Data': [{'Name': 'Sarah', 'Gender': 'Female', 'Schwag': 'Yes', 'Occupation': 'Teacher', 'Occupation Description': 'Educates students', 'Salary': '8000'}, {'Name': 'Michael', 'Gender': 'Male', 'Schwag': 'No', 'Occupation': 'Engineer', 'Occupation Description': 'Designs and builds things', 'Salary': '15000'}, {'Name': 'Emily', 'Gender': 'Female', 'Schwag': 'Yes', 'Occupation': 'Doctor', 'Occupation Description': 'Treats patients', 'Salary': '25000'}]}


In [288]:
my_df

,Name,Gender,Schwag,Occupation,Occupation Description,Salary
0,John,Male,Yes,Developer,Works to develop Large Language Models,10000
1,Jane,Female,No,Nurse,Takes care of patients,5000
2,Kevin,Male,Yes,Founder,Starts and runs a business,20000
3,Sarah,Female,Yes,Teacher,Educates students,8000
4,Michael,Male,No,Engineer,Designs and builds things,15000
5,Emily,Female,Yes,Doctor,Treats patients,25000


In [317]:
res = strict_output(system_prompt = f'''
You are given rows of Tabular Data with missing values denoted as NaN. 
For each row, you are to generate another column 'Is Engineer' with output 'True' or 'False'.
You must not truncate the data.
''', 
        user_prompt = new_df,
        output_format = 
{"Extended Data": "<Extended data with new column 'Is Engineer' for all rows>"})

print(res)
my_df2 = pd.DataFrame(res["Extended Data"])

An exception occurred: Expecting ':' delimiter: line 1 column 262 (char 261)
Current invalid json format: {"Extended Data": [{"Name": "John", "Gender": "Male", "Schwag": "Yes", "Occupation": "Software Engineer", "Occupation Description": "Works to develop Large Language Models", "Salary": "10000", "Is Engineer": "True"}, {"Name": "Jane", "Gender": "Female", "S... "Occupation Description": "Codes and develops software", "Salary": "8000", "Is Engineer": "True"}, {"Name": "David", "Gender": "Male", "Schwag": "No", "Occupation": "Teacher", "Occupation Description": "Educates students", "Salary": "6000", "Is Engineer": "False"}]}
{'Extended Data': [{'Name': 'John', 'Gender': 'Male', 'Schwag': 'Yes', 'Occupation': 'Software Engineer', 'Occupation Description': 'Works to develop Large Language Models', 'Salary': '10000', 'Is Engineer': 'True'}, {'Name': 'Jane', 'Gender': 'Female', 'Schwag': 'No', 'Occupation': 'Nurse', 'Occupation Description': 'Takes care of patients', 'Salary': '5000', 'Is 

# Compare with Original Data

In [318]:
# Original Data
df

,Name,Gender,Schwag,Occupation,Occupation Description,Salary
0,John,Male,Yes,NaN,Works to develop Large Language Models,NaN
1,NaN,Female,No,Nurse,NaN,5000
2,Kevin,NaN,Yes,Founder,NaN,NaN


In [319]:
# Imputed Data Plus Extended Rows and Columns
my_df2

,Name,Gender,Schwag,Occupation,Occupation Description,Salary,Is Engineer
0,John,Male,Yes,Software Engineer,Works to develop Large Language Models,10000,True
1,Jane,Female,No,Nurse,Takes care of patients,5000,False
2,Kevin,Male,Yes,Founder,Starts and runs a business,20000,False
3,Alice,Female,Yes,Software Developer,Codes and develops software,8000,True
4,David,Male,No,Teacher,Educates students,6000,False
